In [1]:
import multiprocessing as mp
import time
import random
from functools import partial
from tqdm import tqdm

import numpy as np
import pandas as pd
from PqiData import *
from PqiDataSdk import *

ds = PqiDataSdk(size=1)

In [2]:
ticker = '000001'
start_date = '20170101'
end_date = '20220831'

date_list = ds.get_trade_dates(start_date=start_date, end_date=end_date)
tickers = ds.get_ticker_list(date='all')
adj_df = ds.get_eod_history(fields=['AdjFactor'],
                            start_date=start_date,
                            end_date=end_date)['AdjFactor']

In [3]:
mins_data = ds.get_mins_history(tickers=['000001'],
                                start_date='20190102',
                                end_date='20190102',
                                begin_time='93000',
                                end_time='145700')['000001']['20190102']
mins_data['hh_bar'] = np.floor(mins_data.index//30).astype(int)
frame_date = mins_data.groupby('hh_bar',).mean()['Twap']
frame_date = frame_date - frame_date
frame = pd.DataFrame({k: frame_date for k in date_list})

In [4]:
frame

,20170103,20170104,20170105,20170106,20170109,20170110,20170111,20170112,20170113,20170116,...,20220818,20220819,20220822,20220823,20220824,20220825,20220826,20220829,20220830,20220831
hh_bar,,,,,,,,,,,,,,,,,,,,,
0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [5]:
def get_hh_twap_ret_intra(ticker, freq=30):
    time.sleep(0.05)
    mins_data = ds.get_mins_history(tickers=[ticker],
                                    start_date=start_date,
                                    end_date=end_date,
                                    begin_time='93000',
                                    end_time='145700')[ticker]
    twap_date = {}
    for date in mins_data.keys():
        try:
            data = mins_data[date]
            # adj = adj_df.loc[ticker, date]
            data['hh_bar'] = np.floor(data.index//freq).astype(int)
            # twap_date[date] = data.groupby('hh_bar',).mean()['Twap'] * adj
            twap_date[date] = data.groupby('hh_bar',).mean()['Twap']
        except Exception as e:
            pass

    twap = pd.DataFrame(twap_date) + frame
    twap_stack = twap.T.stack(dropna=False)
    twap_ret = twap_stack.diff()/twap_stack.shift()
    return twap_ret

In [6]:
def get_hh_twap_ret(ticker, window=1, freq=30):
    time.sleep(0.05)
    mins_data = ds.get_mins_history(tickers=[ticker],
                                    start_date=start_date,
                                    end_date=end_date,
                                    begin_time='93000',
                                    end_time='145700')[ticker]
    twap_date = {}
    for date in mins_data.keys():
        try:
            data = mins_data[date]
            # adj = adj_df.loc[ticker, date]
            data['hh_bar'] = np.floor(data.index//freq).astype(int)
            # twap_date[date] = data.groupby('hh_bar',).mean()['Twap'] * adj
            twap_date[date] = data.groupby('hh_bar',).mean()['Twap']
        except Exception as e:
            pass

    twap = pd.DataFrame(twap_date) + frame
    twap_ret = (twap.diff(window,axis=1)/twap.shift(window,axis=1)).shift(-window,axis=1)
    twap_ret = twap_ret.T.stack(dropna=False).shift(-1)
    return twap_ret

In [83]:
def get_hh_twap_ret_multi(ticker, windows=[1,], freq=30):
    time.sleep(0.05)
    mins_data = ds.get_mins_history(tickers=[ticker],
                                    start_date=start_date,
                                    end_date=end_date,
                                    begin_time='93000',
                                    end_time='145700')[ticker]
    twap_date = {}
    for date in mins_data.keys():
        try:
            data = mins_data[date]
            # adj = adj_df.loc[ticker, date]
            data['hh_bar'] = np.floor(data.index//freq).astype(int)
            # twap_date[date] = data.groupby('hh_bar',).mean()['Twap'] * adj
            twap_date[date] = data.groupby('hh_bar',).mean()['Twap']
        except Exception as e:
            pass
    
    twap = pd.DataFrame(twap_date) + frame
    dict_twap_ret = {}
    for window in windows:
        twap_ret = (twap.diff(window,axis=1)/twap.shift(window,axis=1)).shift(-window,axis=1)
        twap_ret = twap_ret.T.stack(dropna=False).shift(-1)
        dict_twap_ret[window] = twap_ret
    return dict_twap_ret

计算隔日收益

In [7]:
with mp.Pool(processes = 128) as pool:
    result = list(tqdm(pool.imap(get_hh_twap_ret, tickers), total = len(tickers)))

100%|██████████| 5004/5004 [1:45:34<00:00,  1.27s/it]  


In [8]:
df_ret = pd.DataFrame({tickers[i]: result[i] for i in range(len(result))})

df_ret.index.names=[None, None]

df_ret = df_ret.stack().reset_index().set_index(['level_2','level_0','level_1']).sort_index()

df_ret.index.names=[None, None, None]
df_ret.columns = ['ret']

In [9]:
df_ret.to_parquet("/home/shared/Data/data/shared/low_fre_alpha/zy_shared/gru/data/hfRet_1d_30min_new.parquet")

计算衍生target

In [160]:
def save_target(day=3, check=False):
    df_ret = pd.read_parquet(f"/home/shared/Data/data/shared/low_fre_alpha/zy_shared/gru/data/hfRet_{day}d_30min.parquet")

    df_ret_alpha = df_ret.reset_index().groupby(
        ['level_1', 'level_2'])['ret'].apply(lambda x: x - x.mean())

    df_ret_csnorm = df_ret.reset_index().groupby([
        'level_1', 'level_2'
    ])['ret'].apply(lambda x: (x - x.mean()) / (x.quantile(0.9) - x.quantile(0.1)))

    df_ret_rank = df_ret.reset_index().groupby(['level_1',
                                                'level_2'])['ret'].rank(pct=True)
    
    df_ret['ret'] = df_ret_alpha.values
    df_ret.to_parquet(f"/home/shared/Data/data/shared/low_fre_alpha/zy_shared/gru/data/hfRet_alpha_{day}d_30min.parquet")
    df_ret['ret'] = df_ret_csnorm.values
    df_ret.to_parquet(f"/home/shared/Data/data/shared/low_fre_alpha/zy_shared/gru/data/hfRet_csnorm_{day}d_30min.parquet")
    df_ret['ret'] = df_ret_rank.values
    df_ret.to_parquet(f"/home/shared/Data/data/shared/low_fre_alpha/zy_shared/gru/data/hfRet_rank_{day}d_30min.parquet")

In [162]:
save_target(day=3)

In [166]:
save_target(day=5)

In [167]:
save_target(day=10)

计算日内收益

In [11]:
with mp.Pool(processes = 256) as pool:
    result = list(tqdm(pool.imap(get_hh_twap_ret_intra, tickers), total = len(tickers)))

100%|██████████| 5004/5004 [2:51:41<00:00,  2.06s/it]   


In [12]:
df_ret = pd.DataFrame({tickers[i]: result[i] for i in range(len(result))})

df_ret.index.names=[None, None]

df_ret = df_ret.stack().reset_index().set_index(['level_2','level_0','level_1']).sort_index()

df_ret.index.names=[None, None, None]
df_ret.columns = ['ret']

In [30]:
df_ret.to_parquet("/home/shared/Data/data/shared/low_fre_alpha/zy_shared/gru/data/hfRet_30min_30min_new.parquet")

计算风险调整target

In [44]:
df_ret = pd.read_parquet("/home/shared/Data/data/shared/low_fre_alpha/zy_shared/gru/data/hfRet_30min_30min.parquet")

In [45]:
idx0 = list(df_ret.index.levels[0])

In [46]:
def get_sharpe(ticker):
    lst_temp = []
    try:
        for window in (1, 3, 5, 10):
            df_temp = df_ret.loc[ticker]['ret'].rolling(8*window).mean() / df_ret.loc[ticker]['ret'].rolling(8*window).std()
            df_temp = df_temp.shift(-(8*window+1))
            lst_temp.append(df_temp)
    except:
        print(ticker)
    return tuple(lst_temp)

In [47]:
with mp.Pool(processes = 256) as pool:
    result = list(tqdm(pool.imap(get_sharpe, idx0), total = len(idx0)))

100%|██████████| 4943/4943 [08:15<00:00,  9.97it/s]  


In [66]:
df_temp = pd.DataFrame({idx0[i]: result[i][3] for i in range(len(idx0))})

df_temp = pd.DataFrame(df_temp.unstack(0).unstack(), columns=['ret'])

df_temp[df_temp > 10] = np.nan
df_temp[df_temp < -10] = np.nan

df_temp.to_parquet("/home/shared/Data/data/shared/low_fre_alpha/zy_shared/gru/data/hfRet_sharpe_10d_30min.parquet")